In [1]:
import fastf1
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
import math
from shapely.geometry import LineString
from shapely.affinity import translate
import matplotlib.pyplot as plt
import contextily as cx
import imageio.v2 as imageio
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
session_load = fastf1.get_session(2023, 'Monza', 'R')
session_load.load(telemetry=True, laps=True, weather=False)

monza_track = gpd.read_file("bacinger f1-circuits master circuits/it-1922.geojson") #This is monza
original_centroid = monza_track.geometry.centroid.iloc[0]

req         WARNING 	DEFAULT CACHE ENABLED! (491.49 MB) /Users/flo/Library/Caches/fastf1
core           INFO 	Loading data for Italian Grand Prix - Race [v3.3.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '44', '23', '4', '14', '77', '40', '81', '2', '24', '10', '18', '27', '20', '31', '22']


In [4]:
raw_df = pd.read_csv('static/raw_df.csv')
interpolated_df = pd.read_csv('static/interpolated_df.csv')

In [5]:
def get_corners(session):
    circuit_info = session.get_circuit_info()
    corners_df = circuit_info.corners
    return corners_df

In [6]:
def filter_df_to_corner(data,corner,d_threshold):
    data['Distance_to_corner'] = np.sqrt((data['X'] - get_corners(session_load)['X'].iloc[corner]) ** 2 + (data['Y'] - get_corners(session_load)['Y'].iloc[corner]) ** 2)

    threshold_distance = d_threshold

    filtered_df = data[data['Distance_to_corner'] < threshold_distance]

    return filtered_df

In [7]:
def coordinate_shift(original_centroid, f1_api_coords):
    """This translates the original relative coordinates into longitude and latitude
    original_centroid is the centroid computed from the downloaded track data
    """
    centroid_lon, centroid_lat = (original_centroid.x, original_centroid.y)  


      
    # conversion factors - these are approximations, adjust as necessary  
    # 1 degree of latitude is approximately 111 km, and 1 degree of longitude is approximately 111 km multiplied by the cosine of the latitude  
    km_per_degree_lat = 1 / 111  
    km_per_degree_lon = 1 / (111 * math.cos(math.radians(centroid_lat)))  
    
    # your array of tuples  
    xy_coordinates = f1_api_coords
    
    # convert each tuple in the array  
    lonlat_coordinates = []  
    for y,x in xy_coordinates:  
        lon = centroid_lon + (x / 10000) * km_per_degree_lon  # assuming x, y are in meters  
        lat = centroid_lat + (y / 10000) * km_per_degree_lat  # assuming x, y are in meters  
        lonlat_coordinates.append((lon,lat))  
    


    relative_line = LineString(lonlat_coordinates)
    return relative_line



def shift_centroid(relative_line,original_centroid):
    """This shift the centroid computed"""
    # Calculate the distance to translate in each direction  
    # dx = original_centroid.x - relative_line.centroid.x  
    # dy = original_centroid.y - relative_line.centroid.y  
    dx = -0.004091249607403924
    dy = -0.006398742570105753
    # Shift the LineString  
    shifted_line = translate(relative_line, xoff=dx, yoff=dy)  
    return shifted_line

In [ ]:
def folium_plot(centroid,data,plot_type):
    kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='Esri.WorldImagery', attr="Esri",max_zoom=19,maxNativeZoom = 19)
    drivers = set(data['driver'])
    laps = set(data['LapNumber'])

    for lap in laps:
        for driver in drivers:
            if driver in ['22','31']:
                pass
            else:
                data_ = data[(data['driver'] == driver) & (data['LapNumber'] == lap)]
                coords = [(row['Y'],row['X']) for index,row in data_.iterrows()]
                scaled_down = coordinate_shift(centroid,coords)
                shifted_line = shift_centroid(scaled_down,centroid)


                gdf_ = gpd.GeoDataFrame(geometry=[shifted_line], crs="EPSG:4326")    
                new_projected = gdf_.to_crs(epsg=32632)

                style = {'color': 'black', 'weight': 0.4}  # Adjust weight as needed

                folium.GeoJson(new_projected,style=style).add_to(kat)

In [12]:
def plot_all_drivers_for_lap(plot_test, lap,centroid):
    gdfs = []

    for driver in set(plot_test['driver']):
        data = plot_test[(plot_test['driver'] == driver) & (plot_test['LapNumber'] == lap)]
        coords = [(row['Y'], row['X']) for index, row in data.iterrows()]
        scaled_down = coordinate_shift(centroid, coords)
        shifted_line = shift_centroid(scaled_down, centroid)

        data['shifted_x'] = [x for x, y in shifted_line.coords]
        data['shifted_y'] = [y for x, y in shifted_line.coords]

        gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['shifted_x'], data['shifted_y']), crs="EPSG:4326").reset_index(drop=True)
        df_wm = gdf.to_crs(epsg=3857)
        gdfs.append(df_wm)

    # Concatenate all GeoDataFrames
    all_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    # Plot all drivers on the same plot
    ax = all_gdf.plot(column='Speed', legend=True, figsize=(10, 14), cmap='OrRd', markersize=0.4, alpha=1)
    cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery)
    plt.title(f'Lap {lap} - All Drivers')
    plt.tight_layout()
    plt.savefig(f'img/{lap}Lap.png')
    plt.close()

In [13]:
plot_all_drivers_for_lap(filter_df_to_corner(interpolated_df,1,1000),1,original_centroid)

In [14]:
def plot_all_laps_all_drivers(plot_test, centroid):
    # List to store GeoDataFrames for each lap and driver
    gdfs = []

    for lap in set(plot_test['LapNumber']):
        for driver in set(plot_test['driver']):
            data = plot_test[(plot_test['driver'] == driver) & (plot_test['LapNumber'] == lap)]
            coords = [(row['Y'], row['X']) for index, row in data.iterrows()]
            scaled_down = coordinate_shift(centroid, coords)
            shifted_line = shift_centroid(scaled_down, centroid)

            data['shifted_x'] = [x for x, y in shifted_line.coords]
            data['shifted_y'] = [y for x, y in shifted_line.coords]

            gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['shifted_x'], data['shifted_y']), crs="EPSG:4326").reset_index(drop=True)
            df_wm = gdf.to_crs(epsg=3857)
            gdfs.append(df_wm)

    # Concatenate all GeoDataFrames
    all_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    # Plot all laps and all drivers as points on the same plot
    ax = all_gdf.plot(column='Speed', legend=True, figsize=(10, 14), cmap='OrRd', markersize=0.5, alpha=1)
    cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery)
    plt.title('All Laps - All Drivers')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('img/all/AllLaps_AllDrivers.png')
    plt.close()

In [15]:
plot_all_laps_all_drivers(filter_df_to_corner(interpolated_df,1,1000),original_centroid)

In [ ]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def create_gif(image_folder, gif_name):
    images = []
    for filename in sorted(os.listdir(image_folder),key = natural_sort_key):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            images.append(imageio.imread(os.path.join(image_folder, filename)))
    imageio.mimsave(gif_name, images, duration=500)  # Adjust duration as needed

In [ ]:
image_folder = 'img'
gif_name = 'output.gif'

create_gif(image_folder, gif_name)